# Tutorial: OpenClaw NadFun Agent Prompt Book

이 노트북은 NadFun 펀드를 텔레그램 채팅방 단위로 운영하기 위한 프롬프트 기준서입니다.

역할 분리(고정):
- Relayer (중앙 Next 서버)
- Strategy MoltBot
- Participant MoltBot (Mining + Verification)


## 1) 프로젝트 목적 (NadFun)

- Monad 기반 NadFun 토큰(본딩커브/졸업 이벤트)을 데이터 기반으로 평가
- 참여자 MoltBot들의 교차 검증으로 신뢰도 형성
- FINAL claim -> snapshotHash -> intentHash 흐름으로 실행 근거 고정


In [ ]:
NADFUN_CONTEXT = {
    "network": {"testnet": 10143, "mainnet": 143},
    "market_model": "bonding_curve_then_uniswap_v3",
    "core_focus": [
        "token activity",
        "curve state",
        "liquidity",
        "graduation events",
    ],
}

NADFUN_CONTEXT


## 2) 이슈 기반 방향성 확정

정렬 순서:
1. #2 Canonical hashing + EIP-712
2. #4 ClaimBook, #5 Snapshot, #6 IntentBook
3. #8 Relayer/Aggregator, #9 Indexer/API
4. #10~#13 Agent SDK + bots

핵심 결정:
- 중앙 Next 서버 허용(v0) + 온체인 검증 강제
- Strategy와 Relayer는 분리된 권한/키 사용


In [ ]:
ARCH_DECISIONS = {
    "central_next_server_v0": True,
    "onchain_verification_enforced": True,
    "strategy_separate_from_relayer": True,
    "room_scoped_funds": True,
}

ARCH_DECISIONS


## 3) 펀드 룸 멀티테넌트 키

모든 요청/응답에 아래 키를 고정:
- `fundId`
- `roomId`
- `epochId`
- `claimHash | snapshotHash | intentHash`


In [ ]:
REQUIRED_SCOPE_KEYS = [
    "fundId",
    "roomId",
    "epochId",
]

HASH_KEYS = ["claimHash", "snapshotHash", "intentHash"]

REQUIRED_SCOPE_KEYS, HASH_KEYS


## 4) 프롬프트 파일 로드

실제 운영 프롬프트는 `prompts/kr` 폴더의 파일을 사용합니다.


In [ ]:
from pathlib import Path

PROMPT_DIR = Path.cwd() / 'prompts' / 'kr'
FILES = [
    'base_system.md',
    'relayer_next_server.md',
    'strategy_moltbot.md',
    'participant_moltbot.md',
]

loaded = {}
for name in FILES:
    loaded[name] = (PROMPT_DIR / name).read_text(encoding='utf-8')

{n: len(t) for n, t in loaded.items()}


## 5) 역할별 프롬프트 조합기

`base_system` + 역할 프롬프트를 합쳐 모델 입력을 생성합니다.


In [ ]:
def build_prompt(role_file: str) -> str:
    base = loaded['base_system.md']
    role = loaded[role_file]
    return f"{base}

---

{role}".strip()

relayer_prompt = build_prompt('relayer_next_server.md')
strategy_prompt = build_prompt('strategy_moltbot.md')
participant_prompt = build_prompt('participant_moltbot.md')

len(relayer_prompt), len(strategy_prompt), len(participant_prompt)


## 6) 샘플 입력 페이로드

텔레그램 룸 하나(fund-001)에 대한 v0 샘플입니다.


In [ ]:
sample_strategy_input = {
    "taskType": "propose_intent",
    "fundId": "fund-001",
    "roomId": "telegram-room-abc",
    "epochId": 12,
    "snapshot": {
        "snapshotHash": "0xSNAP",
        "finalized": True,
        "claimCount": 19,
    },
    "marketState": {
        "network": 10143,
        "nadfunCurveState": {"progressBps": 7300},
        "liquidity": {"depthUsd": 54000},
        "volatility": {"1h": 0.12},
    },
    "riskPolicy": {
        "maxNotional": "1000",
        "maxSlippageBps": 80,
        "allowlistTokens": ["0xTOKEN"],
        "allowlistVenues": ["NadFun", "UniswapV3"],
    },
}

sample_strategy_input


## 7) 운영 체크

- fund/room 스코프 혼합 금지
- snapshot 미확정 시 intent 제안 금지
- verifier 중복/만료 서명 제거
- JSON only 출력 강제


In [ ]:
def preflight(payload: dict):
    required = ["fundId", "roomId", "epochId", "taskType"]
    missing = [k for k in required if k not in payload]
    if missing:
        return {"ok": False, "missing": missing}
    return {"ok": True}

preflight(sample_strategy_input)


## 8) 다음 액션

1. #2 해시/서명 확정값으로 프롬프트 문구 잠금
2. #4/#5/#6 함수 시그니처가 확정되면 payload 필드 동결
3. #8/#9 API 구현 시 동일 JSON 스키마를 서버 validator에 연결
